# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

# High Glucose Scenario

## Connect to BioExplorer backend

In [ ]:
from bioexplorer import BioExplorer, MovieMaker, Protein, ParametricMembrane, Cell, Sugars, \
                        Vector2, Vector3, Quaternion

hostname = 'localhost:5000'
resource_folder = '../../tests/test_files/'

be = BioExplorer(hostname)
core = be.core_api()
mm = MovieMaker(be)

version = be.version()
be.reset()
print('Version: ' + str(version))

## Global parameters

In [ ]:
# Scene
scene_size = Vector3(800.0, 800.0, 800.0)

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

# Glycans
add_glycans = False
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False

# Immune system
nb_glucoses = 360000
nb_lactoferrins = 50
nb_defensins = 100
nb_defensins_on_virus = 2

# Cell
nb_receptors=100

## Resources

In [ ]:
cache_folder = resource_folder + 'caches/' + version + '/'
image_folder = resource_folder + 'images/'
pdb_folder = resource_folder + 'pdb/'
rna_folder = resource_folder + 'rna/'
obj_folder = resource_folder + 'obj/'
glycan_folder = pdb_folder + 'glycans/'
membrane_folder = pdb_folder + 'membrane/'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

## Host cell

In [ ]:
def add_cell(
        name, size, height, nb_receptors,
        position=Vector3(), random_seed=1):
    
    nb_lipids = 300000
   
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'], occurences=nb_receptors,
        position=Vector3(0.0, 6.0, 0.0))
    
    membrane = ParametricMembrane(
        sources=[
            membrane_folder + 'segA.pdb',
            membrane_folder + 'segB.pdb',
            membrane_folder + 'segC.pdb',
            membrane_folder + 'segD.pdb'
        ],
        occurences=nb_lipids
    )

    cell = Cell(
        name=name, size=size, extra_parameters=[height],
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL, 
        membrane=membrane, receptor=ace2_receptor,
        random_position_seed=1, random_position_strength=0.025,
        random_rotation_seed=2, random_rotation_strength=0.5
    )
    
    be.add_cell(
        cell=cell, position=position,
        representation=protein_representation,
        random_seed=random_seed)

    if nb_receptors != 0 and add_glycans:
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[53, 90, 103, 322, 432, 690])
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[546])

        indices = [[155, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [730, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, representation=glycan_representation,
                chain_ids=[2, 4], site_indices=[index[0]], rotation=index[1])
            be.add_sugars(o_glycan)

In [ ]:
cell_position = Vector3(4.5, -186.0, 7.0)
cell_height = scene_size.y / 10.0
add_cell(
    name='Cell', nb_receptors=nb_receptors,
    size=Vector2(scene_size.x, scene_size.z), height=cell_height,
    position=cell_position, random_seed=10)
status = be.set_models_visibility(True)

## Utility functions

In [ ]:
def get_receptor_positions(name='Cell'):
    '''Get position of receptors on the membrane'''
    receptor_positions = list()
    for i in range(nb_receptors):
        status = be.get_protein_instance_transformation(
            assembly_name=name,
            name=name + '_' + be.NAME_RECEPTOR,
            instance_index=i
        )
        pos = status['position'].split(',')
        position = [float(pos[0]), float(pos[1]), float(pos[2])]
        receptor_positions.append(position)
    return receptor_positions

receptor_positions = get_receptor_positions()

def drop_virus(name, position):
    '''Drop a virus at a given position'''
    status = be.add_coronavirus(
        name=name,
        resource_folder=resource_folder,
        representation=protein_representation,
        position=Vector3(position[0], position[1], position[2]),
        add_glycans=add_glycans)
    status = be.set_models_visibility(True)
    
def get_virus_position_on_receptor(receptor_index):
    '''Get adjusted position of virus on receptor'''
    pos = receptor_positions[receptor_index].copy()
    pos[0] -= 9.0
    pos[1] += 73.0
    pos[2] += 1.0
    return pos

def focus_on_receptor(receptor_index):
    '''Sets the camera focus on specified receptor'''
    pos = get_receptor_positions()[receptor_index].copy()
    pos[2] -= 100.0
    pos[1] += 20.0
    mm.set_camera(
        origin = pos,
        direction = [0.0, 0.0, 1.0],
        up=[0.0, 1.0, 0.0]
    )
    be.core_api().set_renderer()
    return pos

In [ ]:
'''Coronavirus gizmos. Add a sphere at virus positions'''
radius = 45.0
for receptor_index in [90, 23, 24, 98, 37]:
    pos = get_virus_position_on_receptor(receptor_index)
    print(pos)
    be.add_sphere(
        name='Coronavirus_Gizmo_' + str(receptor_index),
        position=Vector3(pos[0], pos[1], pos[2]),
        radius=radius, color=Vector3(1, 0, 0)
    )

In [ ]:
'''Drop viruses on receptors'''
for receptor_index in [90, 23, 24, 98, 37]:
    pos = get_virus_position_on_receptor(receptor_index)
    drop_virus('Coronavirus_' + str(receptor_index), pos)
be.apply_default_color_scheme(be.SHADING_MODE_DIFFUSE)    

In [ ]:
'''Add sphere on top of receptors'''
count = 0
for pos in receptor_positions:
    be.add_sphere(
        name='Receptor ' + str(count),
        position=Vector3(pos[0], pos[1], pos[2]),
        radius=10.0, color=Vector3(1, 0, 0)
    )
    count += 1

In [ ]:
'''High-glucose scenario viruses'''
high_glucose_virus_positions = [
    [-280.5, -97.0, -82.5],
    [ -70.5, -100.0, 229.5],
    [296.5, -125.0, 225.5],
    [204.5, -100.0, 27.5],
    [54.5, -100.0, -242.5]
]

count = 0
for virus_position in high_glucose_virus_positions:
    be.add_sphere(
        name='Coronavirus ' + str(count),
        position=Vector3(
            virus_position[0],
            virus_position[1],
            virus_position[2]
        ),
        radius=radius
    )    
    count +=1

In [ ]:
'''Identify best receptors according to high-glucose virus positions'''
high_glucose_virus_positions = [
    [-280.5, -97.0, -82.5],
    [ -70.5, -100.0, 229.5],
    [296.5, -125.0, 225.5],
    [204.5, -100.0, 27.5],
    [54.5, -100.0, -242.5]
]

high_glucose_virus_positions = [
    [-250.0, -150.0, -70.0],
    [-50.0, -150.0, 250.0],
    [150.0, -150.0, 50.0],
    [40.0, -150.0, -50.0],
    [60.0, -150.0, -240.0],
    [200.0, -150.0, 300.0]
]

virus_indices = list()
virus_positions = list()

print(nb_receptors)
for virus_position in high_glucose_virus_positions:
    for i in range(nb_receptors):
        receptor_position = receptor_positions[i]
        if abs(virus_position[0] - receptor_position[0]) < 55 and \
           abs(virus_position[2] - receptor_position[2]) < 55:
            print(i)
            virus_indices.append(i)
            virus_positions.append(receptor_position)

print(virus_indices)

In [ ]:
'''Slider used to focus on a specified receptor'''
from ipywidgets import IntSlider
from IPython.display import clear_output
receptor_slider = IntSlider(description='frame', min=0, max=nb_receptors-1)

def update_receptor_slider(args):
    receptor_slider.value = args['new']
    receptor_slider.description = str(focus_on_receptor(receptor_slider.value))

receptor_slider.observe(update_receptor_slider, 'value')
display(receptor_slider)

In [ ]:
'''Slider used to add a sphere on top of a specified receptor'''
from ipywidgets import IntSlider
from IPython.display import clear_output
receptor_slider = IntSlider(
    description='frame', min=0, max=nb_receptors-1)

def update_receptor_slider(args):
    receptor_slider.value = args['new']
    v = receptor_slider.value
    p = receptor_positions[v]
    be.add_sphere(
        name='Receptor ' + str(v),
        position=Vector3(p[0], p[1] + 10, p[2]),
        radius=10.0,
        color=Vector3(0,1,0))
    core.set_renderer()

receptor_slider.observe(update_receptor_slider, 'value')
display(receptor_slider)

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=1, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.exposure = 1.2
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 0.75
params.soft_shadows = 0.0
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = core.set_renderer_params(params)

status = core.clear_lights()
status = core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-0.7,-0.4,-1],
    intensity=1.0, is_visible=False
)